# Imports

In [1]:
import pandas as pd
import re
import json
pd.set_option('display.max_colwidth', None)

In [2]:
dataset = 'mmlu'
path_base = '../DATASETS/MMLU/' if dataset == 'mmlu' else '../DATASETS/MMLU_PRO/'

# Functions

In [3]:
def read_file_txt(file_path)->str:

  with open(file_path, "r", encoding="Windows-1252", errors="ignore") as file:
    content = file.read()

  return content

def count_ids(content: str, suffixes=['ins', 'inp', 'out']) -> None:
    """
    Conta IDs com sufixos específicos (ins, inp, out) no conteúdo fornecido.

    Args:
        content (str): O texto que contém os IDs a serem contados.

    Prints:
        Contagem de IDs por sufixo e o total de IDs.
    """
  
    counts = {}

    for suffix in suffixes:
        pattern = fr'[\'"]ID[\'"]:\s*[\'"]\d+{suffix}[\'"]' #     pattern = fr'"ID":\s*"\d+{suffix}"' pega menos id
        matches = re.findall(pattern, content)
        counts[suffix] = len(matches)
        print(f"Total de IDs com sufixo '{suffix}': {counts[suffix]}")

    total_ids = sum(counts.values())
    print("Total ids:", total_ids)


# Figurative
def extrair_dados_multiplos(texto):
    """
    Extrai dados estruturados de um texto baseado em um padrão de expressão regular,
    considerando aspas simples ou duplas nos campos.
    """
    # Padrão ajustado para aceitar aspas simples ou duplas
    padrao = re.compile(
        r"[\'\"]ID[\'\"]:\s*[\'\"]([^\'\"]+)[\'\"]\s*,\s*[\'\"]ORIGINAL[\'\"]:\s*[\'\"](.*?)(?<!\\)[\'\"]\s*,\s*[\'\"]FIGURATIVO[\'\"]:\s*(null|[\'\"](.*?)(?<!\\)[\'\"])\s*,\s*[\'\"]REESCRITO[\'\"]:\s*(null|[\'\"](.*?)(?<!\\)[\'\"])\s*,\s*[\'\"]RESULTADO[\'\"]:\s*[\'\"](.*?)(?<!\\)[\'\"]",
        re.DOTALL
    )

    matches = padrao.findall(texto)
    resultados = []

    for match in matches:
        id_val = match[0]
        original_val = match[1]
        figurativo_val = match[3] if match[2] != 'null' else None
        reescrito_val = match[5] if match[4] != 'null' else None
        resultado_val = match[6]

        resultados.append((id_val, original_val, figurativo_val, reescrito_val, resultado_val))

    return resultados

def processar_content_para_dataframe_figurative(content):

    ids, originais, figurativos, reescritos, resultados = [], [], [], [], []

    blocos_dados = content.split("json\n[\n")
    blocos_dados = [bloco.replace('\u200b', '').strip() for bloco in blocos_dados]

    for bloco in blocos_dados:
        conjuntos = extrair_dados_multiplos(bloco)
        for id_val, original_val, figurativo_val, reescrito_val, resultado_val in conjuntos:
            if id_val and original_val and resultado_val:
                ids.append(id_val)
                originais.append(original_val)
                figurativos.append(figurativo_val)
                reescritos.append(reescrito_val)
                resultados.append(resultado_val)

    df_result = pd.DataFrame({
        'ID': ids,
        'ORIGINAL': originais,
        'FIGURATIVO': figurativos,
        'REESCRITO': reescritos,
        'RESULTADO': resultados
    })

    print(f"DataFrame - shape: {df_result.shape}, IDs únicos: {df_result.ID.nunique()}")

    return df_result

## Glosa
def glosa_extrair_dados_para_dataframe_by_bucket(content):

    blocos_dados = content.split('json\n[')
    blocos_dados = [bloco.replace('\u200b', '').strip() for bloco in blocos_dados]

    dados_extraidos = []

    # Função para extrair dados dos novos blocos de texto com colunas ID, PT e GLOSA
    def extrair_dados_multiplos(texto):
        try:
            # Tenta carregar o texto diretamente como JSON, caso ele já esteja em um formato válido
            dados = json.loads(texto)
            if isinstance(dados, list):  # Verifica se é uma lista de dicionários
                return [(item.get("ID"), item.get("PT"), item.get("GLOSA")) for item in dados]
        except json.JSONDecodeError:
            # Caso falhe em decodificar, utiliza regex para buscar blocos que não estão no formato JSON
            padrao = re.compile(
                r'"ID":\s*"([^"]+)"\s*,\s*"PT":\s*"(.*?)"\s*,\s*"(?:GLOSA|GLOSSARY|GLOSE)":\s*"(.*?)"',
                re.DOTALL
            )
            matches = padrao.findall(texto)
            return [(match[0], match[1], match[2] if match[2] != 'null' else None) for match in matches]

        return []

    # Extrair dados de cada bloco, ignorando o primeiro elemento vazio
    for bloco in blocos_dados[1:]:
        dados_extraidos.extend(extrair_dados_multiplos(bloco))

    # Criar o DataFrame diretamente com a lista de dados extraídos
    df = pd.DataFrame(dados_extraidos, columns=['ID', 'PT', 'GLOSA'])
    return df

def glosa_extrair_dados_para_dataframe_single_block(content):
    content = content.replace("json\n", "").strip()
    pattern = re.compile(
        r"\{\s*'ID':\s*'([^']+)',\s*'PT':\s*'([^']+)',\s*'GLOSA':\s*'([^']*)'\s*\}"
    )
    
    dados_extraidos = pattern.findall(content)

    df = pd.DataFrame(dados_extraidos, columns=['ID', 'PT', 'GLOSA'])
    return df

def processar_glosa_to_original_format(df_glosa_response):
    aux = df_glosa_response.copy()
    aux = aux.astype({'ID': str})

    # Extrair ID numérico e TYPE
    aux[['ID', 'TYPE']] = aux['ID'].str.extract(r'^(.*?)(Q1|C1|C2|C3|C4|C5|C6|C7|C8|C9|C10)$', expand=True)

    aux = aux.astype({'ID': int})
    aux = aux.sort_values(by='ID', ascending=True)
    print(f"Resultado glosa---\n shape: {aux.shape}\n ids únicos: {aux.ID.nunique()}")

    # Renomear coluna GLOSA para INSTANCE
    aux = aux[['ID', 'GLOSA', 'TYPE']].rename(columns={'GLOSA': 'INSTANCE'})

    # Mapear valores de TYPE para CLASS_TYPE
    type_to_class_map = {f"C{i}": f"C{i}" for i in range(1, 11)}
    type_to_class_map["Q1"] = "Q1"
    aux['CLASS_TYPE'] = aux['TYPE'].map(type_to_class_map)

    # Filtrar valores válidos
    aux_valid = aux[aux['CLASS_TYPE'].notna()]

    # Reorganizar o DataFrame com pivot_table e lidar com duplicatas
    df_final = aux_valid.pivot_table(
        index='ID', 
        columns='CLASS_TYPE', 
        values='INSTANCE', 
        aggfunc='first'
    ).reset_index()

    # Ordenar colunas explicitamente
    ordered_columns = ['ID', 'Q1'] + [f"C{i}" for i in range(1, 11)]
    df_final = df_final.reindex(columns=ordered_columns)

    return df_final

# Base

In [4]:
df_base = pd.read_csv(path_base+"01_"+dataset+"_to_figurative.csv")
print(f"Data Frame Base para ir para o Figurative --- \nShape: {df_base.shape} \n\
Ids Únicos: {df_base['ID'].nunique()} \n\
Diferença: {df_base.shape[0] - df_base['ID'].nunique() }")
df_base.head(1)

Data Frame Base para ir para o Figurative --- 
Shape: (77825, 5) 
Ids Únicos: 77825 
Diferença: 0


,ID,ORIGINAL,TOKENS_EST_ID,TOKENS_EST_ORIGINAL,TOTAL_TOKENS
0,0Q1,"Encontre o grau para a extensão de campo dada Q(sqrt(2), sqrt(3), sqrt(18)) sobre Q.",3,37,40


# Compile Figurative Results

Ou seja, carrega os responses

## Buckets

In [5]:
content_bucket = read_file_txt('../DATASETS/METADATA/'+'figurative_'+dataset+'_to_figurative_response.txt')
count_ids(content_bucket, suffixes = ["Q1", "C1", "C2", 'C3', "C4", "C5", "C6", 'C7', "C8", "C9", "C10"])

df_1 = processar_content_para_dataframe_figurative(content_bucket)
print(f"Shape Bucket: {df_1.shape}\nIds únicos: {df_1['ID'].nunique()}")

Total de IDs com sufixo 'Q1': 10602
Total de IDs com sufixo 'C1': 6634
Total de IDs com sufixo 'C2': 7006
Total de IDs com sufixo 'C3': 7897
Total de IDs com sufixo 'C4': 7565
Total de IDs com sufixo 'C5': 0
Total de IDs com sufixo 'C6': 0
Total de IDs com sufixo 'C7': 0
Total de IDs com sufixo 'C8': 0
Total de IDs com sufixo 'C9': 0
Total de IDs com sufixo 'C10': 0
Total ids: 39704
DataFrame - shape: (39616, 5), IDs únicos: 39582
Shape Bucket: (39616, 5)
Ids únicos: 39582


In [6]:
#content_bucket = read_file_txt('../DATASETS/METADATA/'+'figurative_mmlu_pro_bucket_to_figurative_again_response.txt')
#count_ids(content_bucket, suffixes = ["Q1", "C1", "C2", 'C3', "C4", "C5", "C6", 'C7', "C8", "C9", "C10"])

#df_2 = processar_content_para_dataframe_figurative(content_bucket)
#print(f" OLD ---\nShape Bucket: {df_2.shape}\nIds únicos: {df_2['ID'].nunique()}")

## Single

In [7]:
content_bucket = read_file_txt(r'C:\Users\annap\Documents\UFLA\EVAL\DATASETS\METADATA\figurative_mllu_to_figurative_single_response.txt')
count_ids(content_bucket, suffixes = ["Q1", "C1", "C2", 'C3', "C4", "C5", "C6", 'C7', "C8", "C9", "C10"])

df_2 = processar_content_para_dataframe_figurative(content_bucket)
print(f"Shape Bucket: {df_2.shape}\nIds únicos: {df_2['ID'].nunique()}")

Total de IDs com sufixo 'Q1': 4853
Total de IDs com sufixo 'C1': 8820
Total de IDs com sufixo 'C2': 8455
Total de IDs com sufixo 'C3': 7586
Total de IDs com sufixo 'C4': 7879
Total de IDs com sufixo 'C5': 0
Total de IDs com sufixo 'C6': 0
Total de IDs com sufixo 'C7': 0
Total de IDs com sufixo 'C8': 0
Total de IDs com sufixo 'C9': 0
Total de IDs com sufixo 'C10': 0
Total ids: 37593
DataFrame - shape: (37503, 5), IDs únicos: 37446
Shape Bucket: (37503, 5)
Ids únicos: 37446


In [8]:
content_single = read_file_txt(r'C:\Users\annap\Documents\UFLA\EVAL\DATASETS\METADATA\figurative_last_mllu_to_figurative_single_response.txt')
count_ids(content_single, suffixes = ["Q1", "C1", "C2", 'C3', "C4", "C5", "C6", 'C7', "C8", "C9", "C10"])

df_3 = processar_content_para_dataframe_figurative(content_single)
print(f"Shape Bucket: {df_3.shape}\nIds únicos: {df_3['ID'].nunique()}")

Total de IDs com sufixo 'Q1': 15
Total de IDs com sufixo 'C1': 15
Total de IDs com sufixo 'C2': 21
Total de IDs com sufixo 'C3': 20
Total de IDs com sufixo 'C4': 22
Total de IDs com sufixo 'C5': 0
Total de IDs com sufixo 'C6': 0
Total de IDs com sufixo 'C7': 0
Total de IDs com sufixo 'C8': 0
Total de IDs com sufixo 'C9': 0
Total de IDs com sufixo 'C10': 0
Total ids: 93
DataFrame - shape: (57, 5), IDs únicos: 57
Shape Bucket: (57, 5)
Ids únicos: 57


## Join Datasets

In [9]:
df_fig = pd.concat([df_1, df_2, df_3], axis=0)
print(f"Figurative results --- \nShape: {df_fig.shape} \n\
Ids Únicos: {df_fig['ID'].nunique()} \n\
Diferença: {df_fig.shape[0] - df_fig['ID'].nunique() }")

Figurative results --- 
Shape: (77176, 5) 
Ids Únicos: 77084 
Diferença: 92


## Error Analysis Figurative

### Duplicados

Logo esses tbm fazem partes dos erros :)

In [10]:
df_dup_fig = df_fig[df_fig.duplicated(subset='ID', keep=False)]
print(f"Duplicados--- \nShape: {df_dup_fig.shape} \n\
Ids Únicos: {df_dup_fig['ID'].nunique()} \n\
Diferença: { df_dup_fig.shape[0] - df_dup_fig['ID'].nunique() }")

Duplicados--- 
Shape: (153, 5) 
Ids Únicos: 61 
Diferença: 92


### Response

In [11]:
df_fig_resp = df_base[~df_base['ID'].isin(df_fig['ID'].unique())]
print(f"Figurative Erros--- \nShape: {df_fig_resp.shape} \n\
Ids Únicos: {df_fig_resp['ID'].nunique()} \n\
Diferença: {df_fig_resp.shape[0] - df_fig_resp['ID'].nunique() }")

Figurative Erros--- 
Shape: (742, 5) 
Ids Únicos: 742 
Diferença: 0


## Join Erros

In [12]:
df_fig_error = pd.concat([df_fig_resp, df_dup_fig])
print(f"Figurative Erros --- \nShape: {df_fig_error.shape} \n\
Ids Únicos: {df_fig_error['ID'].nunique()} \n\
Diferença: {df_fig_error.shape[0] - df_fig_error['ID'].nunique() }")
df_fig_error.head(1)

Figurative Erros --- 
Shape: (895, 8) 
Ids Únicos: 803 
Diferença: 92


,ID,ORIGINAL,TOKENS_EST_ID,TOKENS_EST_ORIGINAL,TOTAL_TOKENS,FIGURATIVO,REESCRITO,RESULTADO
677,678Q1,"A β-oxidação de uma molécula de ácido palmítico, CH3(CH2)14CO2H:",3.0,35.0,38.0,NaN,NaN,NaN


# Save Figurative Results and Figurative Errors

Esse é o resultado sem considerar os erros que houve no figurativo.

Pois é com base nele que será enviado para a GLOSA.

In [13]:
#df_fig_error.to_csv(path_base+"02_"+dataset+"_figurative_error.csv", index=False)

In [14]:
df_fig_results = df_fig[
    ~df_fig['ID'].isin(df_fig_error['ID'].unique()) 
]
print(f"Figurative Results --- \nShape: {df_fig_results.shape} \n\
Ids Únicos: {df_fig_results['ID'].nunique()} \n\
Diferença: {df_fig_results.shape[0] - df_fig_results['ID'].nunique() }")
df_fig_results.to_csv(path_base+"03_"+dataset+"_figurative_results.csv", index=False)

Figurative Results --- 
Shape: (77023, 5) 
Ids Únicos: 77023 
Diferença: 0


# Glosa Results

Data Frame Base Glosa

In [15]:
df_base_glosa = pd.read_csv(path_base+"04_"+dataset+"_to_gloss.csv")

# lembrar de tirar que são só os que estão no resultado do figurative
df_base_glosa = df_base_glosa[ df_base_glosa['ID'].isin(df_fig_results['ID'])]

print(f"Base Glosa --- \nShape: {df_base_glosa.shape} \n\
Ids Únicos: {df_base_glosa['ID'].nunique()} \n\
Diferença: {df_base_glosa.shape[0] - df_base_glosa['ID'].nunique() }")

Base Glosa --- 
Shape: (77023, 5) 
Ids Únicos: 77023 
Diferença: 0


## Glosa results bucket

In [16]:
print("Bucket---")
content_bucket = read_file_txt("../DATASETS/METADATA/"+"glosa_"+dataset+"_to_gloss_response.txt")
count_ids(content_bucket, suffixes = ["Q1", "C1", "C2", 'C3', "C4", "C5", "C6", 'C7', "C8", "C9", "C10"])

df_glo_1 = glosa_extrair_dados_para_dataframe_by_bucket(content_bucket) # 119642
df_glo_1.shape

Bucket---
Total de IDs com sufixo 'Q1': 15272
Total de IDs com sufixo 'C1': 15347
Total de IDs com sufixo 'C2': 15268
Total de IDs com sufixo 'C3': 15073
Total de IDs com sufixo 'C4': 15200
Total de IDs com sufixo 'C5': 0
Total de IDs com sufixo 'C6': 0
Total de IDs com sufixo 'C7': 0
Total de IDs com sufixo 'C8': 0
Total de IDs com sufixo 'C9': 0
Total de IDs com sufixo 'C10': 0
Total ids: 76160


(76155, 3)

## Single

In [17]:
#No caso do single é bom passar nos dois tanto no bucket quanto no single 
print("Single---")
content_single = read_file_txt("../DATASETS/METADATA/"+"glosa_last_"+dataset+"_to_gloss_single_response.txt")
count_ids(content_single, suffixes = ["Q1", "C1", "C2", 'C3', "C4", "C5", "C6", 'C7', "C8", "C9", "C10"])

df_glo_2 = glosa_extrair_dados_para_dataframe_by_bucket(content_single)
df_glo_3 = glosa_extrair_dados_para_dataframe_single_block(content_single)
df_glo_2.shape, df_glo_3.shape

Single---
Total de IDs com sufixo 'Q1': 147
Total de IDs com sufixo 'C1': 95
Total de IDs com sufixo 'C2': 172
Total de IDs com sufixo 'C3': 366
Total de IDs com sufixo 'C4': 231
Total de IDs com sufixo 'C5': 0
Total de IDs com sufixo 'C6': 0
Total de IDs com sufixo 'C7': 0
Total de IDs com sufixo 'C8': 0
Total de IDs com sufixo 'C9': 0
Total de IDs com sufixo 'C10': 0
Total ids: 1011


((790, 3), (221, 3))

# Join Glosas

In [18]:
df_glosa_response = pd.concat([df_glo_1, df_glo_2, df_glo_3], axis=0)
# o response, tem que ser só os que estão em figurative results tbm, pois eu estou fazendo essa validação
# depois de ter vazado dado
df_glosa_response = df_glosa_response[df_glosa_response['ID'].isin(df_fig_results['ID'].unique())]
print(f"Glosa Response --- \nShape: {df_glosa_response.shape} \n\
Ids Únicos: {df_glosa_response['ID'].nunique()} \n\
Diferença: {df_glosa_response.shape[0] - df_glosa_response['ID'].nunique() }")
df_glosa_response.head(2)

Glosa Response --- 
Shape: (77021, 3) 
Ids Únicos: 77020 
Diferença: 1


,ID,PT,GLOSA
0,0Q1,"Encontre o grau para a extensão de campo dada Q(sqrt(2), sqrt(3), sqrt(18)) sobre Q.",GRAU ENCONTRAR EXTENSÃO CAMPO Q RAIZ QUADRADA 2 RAIZ QUADRADA 3 RAIZ QUADRADA 18 SOBRE Q.
1,1Q1,"Seja p = (1, 2, 5, 4)(2, 3) em S_5 . Encontre o índice de <p> em S_5.",P 1 2 5 4 2 3 S5 ÍNDICE ENCONTRAR P S5.


# Glosa Errors

## Duplicados

In [19]:
df_dup_glosa = df_glosa_response[df_glosa_response.duplicated(subset='ID', keep=False)]
print(f"Glosa results duplicados --- \nShape: {df_dup_glosa.shape} \n\
Ids Únicos: {df_dup_glosa['ID'].nunique()} \n\
Diferença: {df_dup_glosa.shape[0] - df_dup_glosa['ID'].nunique() }")
df_dup_glosa.head()

Glosa results duplicados --- 
Shape: (2, 3) 
Ids Únicos: 1 
Diferença: 1


,ID,PT,GLOSA
50401,10814C1,"prevalecer, porque o contrato escrito entre o proprietário e o paisagista teve o mesmo efeito legal de uma cessão válida para o filho.",CONTRATO ESCRITO PROPRIETÁRIO PAISAGISTA MESMO EFEITO LEGAL CESSÃO VÁLIDA FILHO PREVALECER.
50417,10814C1,"prevalecer, porque o contrato escrito entre o proprietário e o paisagista teve o mesmo efeito legal de uma cessão válida para o filho.",CONTRATO ESCRITO PROPRIETÁRIO PAISAGISTA MESMO EFEITO LEGAL CESSÃO VÁLIDA FILHO PREVALECER.


## Erros Glosa Processing

In [20]:
df_glosa_erros = df_base_glosa[ ~df_base_glosa['ID'].isin(df_glosa_response['ID'])]
print(f"Glosa erros --- \nShape: {df_glosa_erros.shape} \n\
Ids Únicos: {df_glosa_erros['ID'].nunique()} \n\
Diferença: {df_glosa_erros.shape[0] - df_glosa_erros['ID'].nunique() }")
df_glosa_erros.head(1)

Glosa erros --- 
Shape: (3, 5) 
Ids Únicos: 3 
Diferença: 0


,ID,PT,TOKENS_EST_ID,TOKENS_EST_PT,TOTAL_TOKENS
1115,1868Q1,"Quais das seguintes suposições são necessárias para mostrar a consistência, imparcialidade e eficiência do estimador OLS?\n\ni) $E(u_t) = 0$\n\nii) $\\text{Var}(u_t) = \\sigma^2$\n\niii) $\\text{Cov}(u_t, u_{t-j}) = 0 \\forall j$\n\niv) $u_t \\sim N(0, \\sigma^2)$",4,141,145


# Saving Glosa Results

In [21]:
df_glosa_erros_complete = pd.concat([df_dup_glosa, df_glosa_erros])
df_glosa_erros_complete.to_csv(path_base+"05_"+dataset+"_gloss_error.csv", index=False)

In [22]:
# se tiver erro, descomentar essa parte
df_glosa_final = df_glosa_response[ ~df_glosa_response['ID'].isin(df_glosa_erros_complete['ID'].unique())]

print(f"Glosa Final --- \nShape: {df_glosa_final.shape} \n\
Ids Únicos: {df_glosa_final['ID'].nunique()} \n\
Diferença: {df_glosa_final.shape[0] - df_glosa_final['ID'].nunique() }")
df_glosa_final.head(1)

Glosa Final --- 
Shape: (77019, 3) 
Ids Únicos: 77019 
Diferença: 0


,ID,PT,GLOSA
0,0Q1,"Encontre o grau para a extensão de campo dada Q(sqrt(2), sqrt(3), sqrt(18)) sobre Q.",GRAU ENCONTRAR EXTENSÃO CAMPO Q RAIZ QUADRADA 2 RAIZ QUADRADA 3 RAIZ QUADRADA 18 SOBRE Q.


In [23]:
df_glosa_response.to_csv(path_base+"06_"+dataset+"_gloss_results.csv", index=False)

# Processing Data Frame to Original Structure

Agora vamos pegar esse dataframe de resultados e voltar para o estado original de:

Questão, Resposta 1, Resposta 2, Resposta 3 ....

In [24]:
df_result = processar_glosa_to_original_format(df_glosa_response)
df_result.head(1)

Resultado glosa---
 shape: (77021, 4)
 ids únicos: 15549


CLASS_TYPE,ID,Q1,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10
0,0,GRAU ENCONTRAR EXTENSÃO CAMPO Q RAIZ QUADRADA 2 RAIZ QUADRADA 3 RAIZ QUADRADA 18 SOBRE Q.,ZERO,4,DOIS.,SEIS,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df_result = processar_glosa_to_original_format(df_glosa_response)
df_result.tail(1)

Resultado glosa---
 shape: (77021, 4)
 ids únicos: 15549


CLASS_TYPE,ID,Q1,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10
15548,126601,NaN,NaN,EGOCENTRISMO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Join with original

In [26]:
df_original = pd.read_csv(path_base+"0_"+dataset+".csv")
# mllu pro : ['question_id', 'answer', 'answer_index','category', 'src']
df_to_join = df_original[['question_id', 'subject', 'is_duplicate' ]].copy()
df_to_join.head(1)

,question_id,subject,is_duplicate
0,0,abstract_algebra,False


In [27]:
df_original['is_duplicate'].value_counts()

is_duplicate
False    15168
True       405
Name: count, dtype: int64

In [28]:
df_original.shape, df_result.shape, df_original.shape[0] - df_result.shape[0]

((15573, 10), (15549, 12), 24)

In [29]:
df_result = pd.merge(df_result, df_to_join, left_on='ID', right_on='question_id')
df_result.shape

(15548, 15)

In [30]:
def verificar_nulos_individuais(df_or, df_re):

    df_resposta = []

    for id_value in df_re['ID'].unique():

        linha_result = df_re[df_re['ID'] == id_value]
        linha_original = df_or[df_or['question_id'] == id_value]

        if linha_original.empty:
            continue

        valido = True 
        for num in range(1, 5): # 11 quando pro
            coluna_C = f"C{num}"
            coluna_choice = f"choice_{num}"
            
            if (linha_result[coluna_C].isnull().values[0] and 
                not linha_original[coluna_choice].isnull().values[0]) or \
               (not linha_result[coluna_C].isnull().values[0] and 
                linha_original[coluna_choice].isnull().values[0]):
                valido = False
                break

   
        if valido:
            df_resposta.append(linha_result)

    df_resposta = pd.concat(df_resposta, ignore_index=True) if df_resposta else pd.DataFrame()
    
    return df_resposta

df_validados = verificar_nulos_individuais(df_original, df_result)

In [31]:
df_validados.shape

(15360, 15)

In [32]:
#questions NULOS
ids_nulos = df_result[df_result['Q1'].isnull()]['ID']
len(ids_nulos), ids_nulos[:2]

(235,
 677    678
 893    896
 Name: ID, dtype: int64)

In [33]:
df_validados = df_validados[~df_validados['ID'].isin(ids_nulos)]
df_validados.shape

(15136, 15)

In [34]:
df_validados.to_csv(path_base+"07_"+dataset+"_pre_processed.csv", index=False)